<a href="https://colab.research.google.com/github/jiahuanluo/text_matching/blob/main/text_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
!pip install tokenizers
!pip install transformers

import transformers
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/Colab')

     |████████████████████████████████| 3.2MB 8.5MB/s 
     |████████████████████████████████| 2.0MB 8.2MB/s 
     |████████████████████████████████| 890kB 35.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=90b090f8dab8f01e08485a6caa6646a9a9fced219d1d619c718911af64309e51
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Mounted at /content/drive


In [ ]:
max_len = 0
train_file = open("./data/train.txt", "w")
with open('./data/gaiic_track3_round1_train_20210228.tsv') as f:
  lines = f.readlines()
  for line in lines:
    line = line.strip().split('\t')[:2]
    # line = ["[" + s +"]" for s in " ".join(line).split()]
    line = " ".join(line)
    if len(line.split()) > max_len:
      max_len = len(line.split())
    train_file.write(line + "\n")
  f.close()
with open('./data/gaiic_track3_round1_testA_20210228.tsv') as f:
  lines = f.readlines()
  for line in lines:
    line = line.strip().split('\t')[:2]
    # line = ["[" + s +"]" for s in " ".join(line).split()]
    line = " ".join(line)
    if len(line.split()) > max_len:
      max_len = len(line.split())
    train_file.write(line + "\n")
  f.close()
train_file.close()
max_len

90

In [ ]:
import tokenizers

# bwpt = tokenizers.CharBPETokenizer()
 
# filepath = root + 'train.txt'

# bwpt.train(
#     files=[filepath],
#     min_frequency=3,

# ) 


from pathlib import Path

from tokenizers import CharBPETokenizer

paths = ['./data/train.txt']

# Initialize a tokenizer
tokenizer = CharBPETokenizer(split_on_whitespace_only=True)

# Customize training
tokenizer.train(files=paths, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


tokenizer.save_model('./output')
vocab = tokenizer.get_vocab()

In [ ]:
vocab['<pad>']

1

In [ ]:
tokenizer.encode('1 2 3 4 5 6 7 8 9 10 4 11').tokens

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer, CharBPETokenizer
from tokenizers.processors import BertProcessing


tokenizer = CharBPETokenizer(
    "./output/vocab.json",
    "./output/merges.txt",
)

In [ ]:
tokenizer

Tokenizer(vocabulary_size=12531, model=BPE, unk_token=<unk>, suffix=</w>, dropout=None, lowercase=False, unicode_normalizer=None, bert_normalizer=True, split_on_whitespace_only=False)

In [ ]:
sentence_1 = '1 2 3 4 5 6 7'
sentence_2 = '8 9 10 4 11'
tokenizer.encode([sentence_1, sentence_2])

TypeError: ignored

In [ ]:
!nvidia-smi

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=12531,
    max_position_embeddings=130,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)


In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./output", max_len=128)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)
model.num_parameters()

52857843

In [ ]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./data/train.txt",
    block_size=128,
)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


CPU times: user 9.05 s, sys: 365 ms, total: 9.41 s
Wall time: 6.77 s


In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
trainer.train()
trainer.save_model("./output")

In [ ]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('./output')
model.train()

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [ ]:
# from tokenizers.implementations import ByteLevelBPETokenizer, CharBPETokenizer
# from tokenizers.processors import BertProcessing


# tokenizer = CharBPETokenizer(
#     "./output/vocab.json",
#     "./output/merges.txt",
# )
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('./output')
text_batch = ["1 2 3 4 11", "5 6 7 8 9 10"]
encoding = tokenizer(text_batch, return_tensors='pt', padding=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
voc = tokenizer.get_vocab()

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
from torch.utils.data import Dataset, DataLoader

class PairDataset(Dataset):
    def __init__(self, file_path):
        self.file_path = file_path
        self.lines = open(self.file_path).readlines()

    def __getitem__(self, index):
        line = self.lines[index]
        line = line.strip().split("\t")
        X = " ".join(line[:2])
        y = int(line[-1])
        return X, y
    def __len__(self):
        return len(self.lines)

In [ ]:
train_file = "./data/gaiic_track3_round1_train_20210228.tsv"
train_data = PairDataset(train_file)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
total_loss = 0
for epoch in range(20):
  for i, (x, y) in enumerate(train_loader):
    x = tokenizer(list(x), return_tensors='pt', padding=True)
    labels = y.unsqueeze(0)
    outputs = model(x['input_ids'], attention_mask=x['attention_mask'], labels=labels)
    loss = outputs.loss
    loss.backward()
    total_loss += loss.item()
    optimizer.step()
    if i % 10 == 0:
      print("Epoch: {} | Iter: {} | Loss: {:.3f}".format(epoch, i, total_loss/10))
      total_loss = 0


Epoch: 0 | Iter: 0 | Loss: 0.067
Epoch: 0 | Iter: 10 | Loss: 0.674
Epoch: 0 | Iter: 20 | Loss: 0.639
Epoch: 0 | Iter: 30 | Loss: 0.664
Epoch: 0 | Iter: 40 | Loss: 0.678
Epoch: 0 | Iter: 50 | Loss: 0.690
Epoch: 0 | Iter: 60 | Loss: 0.702
Epoch: 0 | Iter: 70 | Loss: 0.736
Epoch: 0 | Iter: 80 | Loss: 0.783
Epoch: 0 | Iter: 90 | Loss: 0.822
Epoch: 0 | Iter: 100 | Loss: 0.878
Epoch: 0 | Iter: 110 | Loss: 0.906
Epoch: 0 | Iter: 120 | Loss: 0.950
Epoch: 0 | Iter: 130 | Loss: 0.939
Epoch: 0 | Iter: 140 | Loss: 0.987
Epoch: 0 | Iter: 150 | Loss: 1.068


KeyboardInterrupt: ignored

In [ ]:
from torch.utils.data import Dataset, DataLoader
class PairDataset(Dataset):
  def __init__(self)